In [1]:
import pandas as pd
terminal_profiles_table=pd.read_pickle("../data/raw/temrinal.pkl")

In [2]:
terminal_profiles_table.head()

,TERMINAL_ID,x_terminal_id,y_terminal_id
0,0,41.702200,72.032449
1,1,0.011437,30.233257
2,2,14.675589,9.233859
3,3,18.626021,34.556073
4,4,39.676747,53.881673


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField

In [4]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("data_store")
         .getOrCreate())

25/02/13 21:25:26 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
25/02/13 21:25:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d598b265-71be-4a2e-8b1b-6f320282f98c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [5]:
terminal_sdf=spark.createDataFrame(terminal_profiles_table)

In [6]:
# Rename column 'name' to 'full_name'
#df_renamed = df.withColumnRenamed("name", "full_name")

# Show the result
#df_renamed.show()

terminal_sdf=terminal_sdf.withColumnRenamed("TERMINAL_ID","terminal_id")

terminal_sdf=terminal_sdf.withColumnRenamed("x_terminal_id","x_location")
terminal_sdf=terminal_sdf.withColumnRenamed("y_terminal_id","y_location")

In [7]:
from pyspark.sql.functions import current_timestamp
terminal_sdf=terminal_sdf.withColumn("row_created_timestamp", current_timestamp())
terminal_sdf=terminal_sdf.withColumn("row_updated_timestamp", current_timestamp())

In [8]:
terminal_sdf.createOrReplaceTempView("terminal_sdf")

25/02/13 21:25:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [9]:
spark.sql("""
        CREATE TABLE IF NOT EXISTS payment.terminal (
      terminal_id INT,
      x_location FLOAT,
      y_location FLOAT,
      row_created_timestamp TimeStamp,
      row_updated_timestamp TimeStamp
)
USING iceberg
""")

DataFrame[]

In [10]:
spark.sql("""
MERGE INTO payment.terminal AS target
USING terminal_sdf AS source
ON target.terminal_id = source.terminal_id
WHEN MATCHED THEN
  UPDATE SET
    target.x_location = source.x_location,
    target.y_location = source.y_location,
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    terminal_id,
    x_location,
    y_location,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.terminal_id,
    source.x_location,
    source.y_location,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""")

DataFrame[]